In [1]:
import pandas as pd
import implicit
import os
import numpy as np
from scipy import sparse

Решается задача товарных рекомендаций.

- `purchases_train.csv` - история покупок в розничном магазине (с 21 октября 2003 года по 12 марта 2004 года)

- `purchases_test.csv` - покупки за следующую неделю (с 13 по 19 марта 2004 года). В этой выборке для каждого пользователя исключены товары, которые он уже покупал за период обучающей выборки

- `customers.csv` - пол клиентов

В решении ниже
- обучается модель матричного разложения AlternateLeastSquares
- сравнивается с тестовыми данными
- измеряется ее качество по метрике map@10
- данные о поле клиентов не используются

Улучшите решение с помощью информацию о поле клиентов

In [2]:
purchases_train = pd.read_csv('purchases_train.csv')
purchases_train.head()

customer_id  product_id                       datetime
0      8698595       12530  2004-03-10 22:18:43.497459200
1     13271885        7541  2004-03-06 02:24:43.209763200
2     16852746       13134  2004-03-10 01:03:09.598614400
3     16852746        6572  2004-03-04 16:45:16.522566400
4     14619070        4659  2004-03-12 13:29:35.011481600

Обучаем модель AlternateLeastSquares

In [3]:
user_items = sparse.coo_matrix(
    (
        np.ones(purchases_train.customer_id.size, dtype=np.float32),
        (
            purchases_train.customer_id,
            purchases_train.product_id
        )
    )
).tocsr()

In [4]:
item_users = user_items.T.tocsr()

In [5]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100)

In [6]:
np.random.seed(42)
model.fit(item_users=item_users)

100%|██████████████████████████████████████████████████████████████████████████████| 100.0/100 [01:04<00:00,  1.60it/s]


purchases_test.csv содержит данные о покупках с 13 марта 2004 по 20 марта 2004 - то есть неделя следующая за обучающей выборкой

для каждого пользователя исключены те товары, которые он покупал в обучающей выборке

In [7]:
purchases_test = pd.read_csv('purchases_test.csv')
display(
    purchases_test.head(),
)

customer_id  product_id                       datetime
0      1021292        6197  2004-03-18 13:35:19.145152000
1     11379978        4659  2004-03-19 18:51:31.887936000
2     13271885        5659  2004-03-14 05:47:21.544166400
3     13271885        1015  2004-03-15 14:41:19.702089601
4     12315337       12072  2004-03-19 10:39:17.148105600

Измеряем качество рекомендаций с помощью метрики map@10

In [8]:
relevant = purchases_test.groupby('customer_id')['product_id'].apply(lambda s: s.values).reset_index()
relevant.rename(columns={'product_id': 'product_ids'}, inplace=True)
relevant.head()

customer_id    product_ids
0          107         [5868]
1          453        [11854]
2         1011  [10609, 7110]
3         1135         [8994]
4         2947         [5868]

In [9]:
recommendations = []
for user_id in relevant['customer_id']:
    recommendations.append([x[0] for x in model.recommend(userid=user_id, user_items=user_items, N=10)])

In [10]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if len(actual) == 0:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [11]:
mapk(relevant['product_ids'], recommendations, k=10)

0.1041872886128091

Задание: используйте пол клиентов для улучшения модели

In [12]:
customers = pd.read_csv('customers.csv')
customers.head()

customer_id     sex
0     14386819  Female
1      1481405     NaN
2     16745074     NaN
3     10325906     NaN
4     11167384     NaN